In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [16]:
import sys
sys.path.insert(0,'/content/head-qa-afi/code')
sys.path.insert(0,'/content/head-qa-afi/code')

In [ ]:
# !git clone https://github.com/claudiaqw/head-qa-afi.git

In [42]:
!pip install transformers
!pip install datasets
!python -m spacy download es_core_news_sm
!pip install pickle5
# !python -m spacy download es_core_news_lg

     |████████████████████████████████| 16.2 MB 7.7 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')
     |████████████████████████████████| 132 kB 8.0 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219295 sha256=72da7c63516d5ea1308523f5c03d62efba847623186612f79fc3f6540d624f36
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5


In [60]:
import os
import time
import datetime
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import copy
import spacy
import pickle5 as pickle
import collections
from tqdm import tqdm_notebook, trange


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, TensorDataset, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import transformers
from transformers.optimization import AdamW
from transformers import BertForSequenceClassification, BertConfig, BertTokenizer, BertModel, BertForMaskedLM
from transformers import get_linear_schedule_with_warmup


%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
BASE_BERT = 'dccuchile/bert-base-spanish-wwm-cased'

In [25]:
from datasets import load_dataset

data_es = load_dataset('head_qa', 'es' )
training, validation, testing = data_es['train'], data_es['validation'], data_es['test']

Reusing dataset head_qa (/root/.cache/huggingface/datasets/head_qa/es/1.1.0/d6803d1e84273cdc4a2cf3c5102945d166555f47b299ecbc5266d582f408f8e2)


In [43]:
import pickle5 as pickle

def load_dataset_from_pickle(filename):
    with open(filename, 'rb') as handle:
        return pickle.load(handle)

def filter_by_category(dataset, category):
    filtered_dataset = []
    for instance in dataset:
        categ = instance['category']
        if categ == category:
            filtered_dataset.append(instance)
    return filtered_dataset

In [54]:
import training
from training import get_optimizer, train, train_ir, validate, validate_ir, evaluator, evaluator_ir, evaluate
from training import load_embeddings_from_file, make_embedding_matrix
from training import pad_seq, encoder_bert, encoder_bert_ir, encoder_bert_instance, encoder_bert_ir_instance
from training import evaluator_bert, evaluator_bert_ir

from ir_models import BERT_QA

In [46]:
training_instances = load_dataset_from_pickle('/content/head-qa-afi/data/training_ir.pickle')
validation_instances = load_dataset_from_pickle('/content/head-qa-afi/data/validation_ir.pickle')
testing_instances = load_dataset_from_pickle('/content/head-qa-afi/data/testing_ir.pickle')
oversampled_training = load_dataset_from_pickle('/content/head-qa-afi/data/oversampled_training_ir.pickle')

In [47]:
tokenizer = BertTokenizer.from_pretrained(BASE_BERT, do_lower_case=False)

In [50]:
train_inputs_0, train_masks_0, train_inputs_1, train_masks_1, train_labels = encoder_bert_ir(oversampled_training, tokenizer)
valid_inputs_0, valid_masks_0, valid_inputs_1, valid_masks_1, valid_labels = encoder_bert_ir(validation_instances, tokenizer)
test_inputs_0, test_masks_0, test_inputs_1, test_masks_1, test_labels = encoder_bert_ir(testing_instances, tokenizer)

In [51]:
train_inputs_0 = torch.tensor(train_inputs_0)
valid_inputs_0 = torch.tensor(valid_inputs_0)
test_inputs_0 = torch.tensor(test_inputs_0)

train_masks_0 = torch.tensor(train_masks_0)
valid_masks_0 = torch.tensor(valid_masks_0)
test_masks_0 = torch.tensor(test_masks_0)

train_inputs_1 = torch.tensor(train_inputs_1)
valid_inputs_1 = torch.tensor(valid_inputs_1)
test_inputs_1 = torch.tensor(test_inputs_1)

train_masks_1 = torch.tensor(train_masks_1)
valid_masks_1 = torch.tensor(valid_masks_1)
test_masks_1 = torch.tensor(test_masks_1)

train_labels = torch.tensor(train_labels)
valid_labels = torch.tensor(valid_labels)
test_labels = torch.tensor(test_labels)

In [52]:
batch_size = 8

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs_0, train_masks_0, train_inputs_1, train_masks_1, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
valid_data = TensorDataset(valid_inputs_0, valid_masks_0, valid_inputs_1, valid_masks_1, valid_labels)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_size)

# Create the DataLoader for our test set.
test_data = TensorDataset(test_inputs_0, test_masks_0, test_inputs_1, test_masks_1, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [57]:
model = BERT_QA(pretrained_model=BASE_BERT)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.we

In [58]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 203 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (31002, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [61]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [63]:
def flat_accuracy(preds, labels):
    pred_flat = np.round(preds, axis=0)
    labels_flat = labels
    print(pred_flat, labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [64]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [65]:
def train_model(model, train_dataloader, valid_dataloader, epochs):
    import random
    seed_val = 42
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    # Store the average loss after each epoch so we can plot them.
    loss_values = []
    epochs_results = []

    for epoch_i in range(0, epochs):
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()
        total_loss = 0
        model.train()
        for step, batch in enumerate(train_dataloader):
            if step % 40 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            b_input_ids_0, b_input_mask_0 = batch[0].long(), batch[1]
            b_input_ids_1, b_input_mask_1 = batch[2].long(), batch[3]
            b_labels = batch[4]
            model.zero_grad()
            out = model(input_ids_0=b_input_ids_0, 
                      attention_mask_0=b_input_mask_0, 
                      input_ids_1=b_input_ids_1,
                      attention_mask_1=b_input_mask_1,
                      labels=b_labels)
            b_labels = b_labels.unsqueeze(dim=1)
            loss = F.binary_cross_entropy(out, b_labels.float())
            total_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
        avg_train_loss = total_loss / len(train_dataloader)  
        loss_values.append(avg_train_loss)
        valid_acc, y_real, y_pred = valid_model(model, valid_dataloader)
        p, r, f1 = evaluate(y_real, y_pred)
        epochs_results.append([avg_train_loss, valid_acc, p, r, f1])

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
    print("")
    print("Training complete!")
    return epochs_results

In [66]:
def valid_model(model, validation_dataloader):
    print("Running Validation...")    
    y_true = []
    y_pred = []
    
    t0 = time.time()
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    
    for batch in validation_dataloader:
        b_input_ids_0, b_input_mask_0 = batch[0].long(), batch[1]
        b_input_ids_1, b_input_mask_1 = batch[2].long(), batch[3]
        with torch.no_grad(): 
            out = model(input_ids_0=b_input_ids_0, 
                      attention_mask_0=b_input_mask_0, 
                      input_ids_1=b_input_ids_1,
                      attention_mask_1=b_input_mask_1,
                      labels=b_labels)
        logits = out
        #logits = logits.detach().cpu().numpy()
        #label_ids = b_labels.to('cpu').numpy()
        tmp_eval_accuracy = flat_accuracy(np.array(logit), np.array(b_labels))
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
        pred = torch.max(logits, dim=1)[1]
        y_true.append(b_labels)
        y_pred.append(pred)        
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    return eval_accuracy/nb_eval_steps, y_true, y_pred

In [67]:
epochs_results = train_model(model, train_dataloader, valid_dataloader, epochs=4)


======== Epoch 1 / 4 ========
Training...
  Batch    40  of  2,657.    Elapsed: 0:01:23.
  Batch    80  of  2,657.    Elapsed: 0:02:46.
  Batch   120  of  2,657.    Elapsed: 0:04:09.


KeyboardInterrupt: ignored

In [ ]:
acc, points = evaluate(model, validation, encoder_bert, evaluator_bert)

acc, points = evaluate(model, testing, encoder_bert, evaluator_bert)
acc, points